In [1]:
import os
import torch
from docling_core.types.doc import DoclingDocument
from docling_core.types.doc.document import DocTagsDocument
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

/home/lingjiebao/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
processor = AutoProcessor.from_pretrained("ds4sd/SmolDocling-256M-preview")
model = AutoModelForVision2Seq.from_pretrained(
    "ds4sd/SmolDocling-256M-preview",
    torch_dtype=torch.bfloat16,
    #_attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
).to(DEVICE)

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Convert this page to docling."}
        ]
    },
]

In [ ]:
from PIL import Image

In [ ]:
image = Image.open("Snipaste_2025-04-20_20-07-12.png")

In [ ]:
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image], return_tensors="pt")
inputs = inputs.to(DEVICE)

In [5]:
import torch

from docling_core.types.doc import DoclingDocument

from docling_core.types.doc.document import DocTagsDocument

from transformers import AutoProcessor, AutoModelForVision2Seq

from transformers.image_utils import load_image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load images

# Initialize processor and model

processor = AutoProcessor.from_pretrained("ds4sd/SmolDocling-256M-preview")

model = AutoModelForVision2Seq.from_pretrained(

    "ds4sd/SmolDocling-256M-preview",

    torch_dtype=torch.bfloat16,

    # _attn_implementation="flash_attention_2"# if DEVICE == "cuda" else "eager",

).to(DEVICE)

model.device

# Load images

image = load_image("https://user-images.githubusercontent.com/12294956/47312583-697cfe00-d65a-11e8-930a-e15fd67a5bb1.png")

# Create input messages

messages = [

    {

        "role": "user",

        "content": [

            {"type": "image"},

            {"type": "text", "text": "Convert this page to docling."}

        ]

    },

]

# Prepare inputs

prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

inputs = processor(text=prompt, images=[image], return_tensors="pt")

inputs = inputs.to(DEVICE)

# Generate outputs

generated_ids = model.generate(**inputs, max_new_tokens=8192)

prompt_length = inputs.input_ids.shape[1]

trimmed_generated_ids = generated_ids[:, prompt_length:]

doctags = processor.batch_decode(

    trimmed_generated_ids,

    skip_special_tokens=False,

)[0].lstrip()

# Populate document

doctags_doc = DocTagsDocument.from_doctags_and_image_pairs([doctags], [image])

print(doctags)

# create a docling document

doc = DoclingDocument(name="Document")

doc.load_from_doctags(doctags_doc)

from IPython.display import display, Markdown

display(Markdown(doc.export_to_markdown()))

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


ValueError: `resolution_max_side` cannot be larger than `max_image_size`